In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import os
from textblob import TextBlob
from nltk.stem import PorterStemmer
from textblob import Word
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sklearn.feature_extraction.text as text
from sklearn import model_selection, preprocessing,linear_model, naive_bayes, metrics, svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from io import StringIO
import seaborn as sns

In [2]:
dataReviews = pd.read_csv('Reviews.csv')
dataReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [3]:
dataReviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
dataReviews["Text"] = dataReviews['Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
dataReviews["Text"].head()

0    i have bought several of the vitality canned d...
1    product arrived labeled as jumbo salted peanut...
2    this is a confection that has been around a fe...
3    if you are looking for the secret ingredient i...
4    great taffy at a great price. there was a wide...
Name: Text, dtype: object

In [5]:
stop = stopwords.words('english')
len(stop)

179

In [ ]:
dataReviews['Text'] = dataReviews['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
dataReviews['Text'].head()

In [ ]:

dataReviews.Score.hist(bins = 5, grid = False)
dataReviews['Score'].value_counts()

In [ ]:
score_1 = dataReviews[dataReviews['Score'] == 1].sample(n = 29743)
score_2 = dataReviews[dataReviews['Score'] == 2].sample(n = 29743)
score_3 = dataReviews[dataReviews['Score'] == 3].sample(n = 29743)
score_4 = dataReviews[dataReviews['Score'] == 4].sample(n = 29743)
score_5 = dataReviews[dataReviews['Score'] == 5].sample(n = 29743)


In [ ]:
reviews_sample = pd.concat([score_1,score_2,score_3,score_4,score_5],axis=0)


In [ ]:
from wordcloud import WordCloud
from wordcloud import STOPWORDS
reviews_str = reviews_sample.Summary.str.cat()
wordcloud = WordCloud(background_color='white').generate(reviews_str)
plt.figure(figsize=(10,10))
plt.imshow(wordcloud,interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
negative_reviews = reviews_sample[reviews_sample['Score'].isin([1, 2])]
positive_reviews = reviews_sample[reviews_sample['Score'].isin([4, 5])]
negative_reviews_str = negative_reviews.Summary.str.cat()
positive_reviews_str = positive_reviews.Summary.str.cat()

In [ ]:
wordcloud_negative = WordCloud(background_color='white').generate(negative_reviews_str)
wordcloud_positive = WordCloud(background_color='white').generate(positive_reviews_str)


In [ ]:
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(211)
ax1.imshow(wordcloud_positive,interpolation='bilinear')
ax1.axis("off")
ax1.set_title('Reviews with Positive Scores',fontsize=20)
plt.show()


fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(211)
ax1.imshow(wordcloud_negative,interpolation='bilinear')
ax1.axis("off")
ax1.set_title('Reviews with Negative Scores',fontsize=20)
plt.show()

In [ ]:
rating ={1:'negative', 2:'neutral', 3:'positive'}
dataReviews['outcome'] = np.zeros(len(dataReviews['Score']))
for i in range(len(dataReviews['Score'])):
    if dataReviews['Score'].iloc[i] > 3:
        dataReviews['outcome'].iloc[i] = 'positive'
    elif dataReviews['Score'].iloc[i] == 3:
        dataReviews['outcome'].iloc[i] = 'neutral'
    else:
        dataReviews['outcome'].iloc[i] = 'negative'
        
        

In [ ]:

dataReviews.head()

In [26]:
dataReviews.to_csv("review_label.csv")

In [33]:
data = pd.read_csv("review_label.csv")
data = data[['Text', 'outcome']]


In [34]:
# data= data.sample(200000)
# data

In [35]:
x_train, x_test, y_train, y_test = train_test_split(data['Text'], data['outcome'])

In [36]:
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [37]:
encoder.inverse_transform(y_train), y_train

(array(['positive', 'positive', 'positive', ..., 'positive', 'neutral',
        'positive'], dtype=object),
 array([2, 2, 2, ..., 2, 1, 2]))

In [38]:
encoder.inverse_transform(y_test), y_test

(array(['positive', 'positive', 'positive', ..., 'positive', 'positive',
        'positive'], dtype=object),
 array([2, 2, 2, ..., 2, 2, 2]))

In [39]:
tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(data['Text'])


TfidfVectorizer(max_features=5000, token_pattern='\\w{1,}')

In [40]:
xtrain_tfidf =  tfidf_vect.transform(x_train)
xtest_tfidf =  tfidf_vect.transform(x_test)

In [41]:
model = linear_model.LogisticRegression().fit(xtrain_tfidf, y_train)


/home/vynguyen/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
accuracy = metrics.accuracy_score(model.predict(xtest_tfidf),y_test)
accuracy

0.8664241383677892

In [43]:
print(metrics.classification_report(y_test, model.predict(xtest_tfidf),target_names=data['outcome'].unique()))

              precision    recall  f1-score   support

    positive       0.74      0.67      0.70     20542
    negative       0.52      0.18      0.27     10675
     neutral       0.90      0.97      0.93    110897

    accuracy                           0.87    142114
   macro avg       0.72      0.61      0.63    142114
weighted avg       0.85      0.87      0.85    142114



In [66]:
# pd.set_option('display.max_colwidth', None)
# data.head()
text = ['the noddles is very bad']
text_features = tfidf_vect.transform(text)
predictions = model.predict(text_features)
predictions

array([0])

In [67]:
encoder.inverse_transform(y_train), y_train

(array(['positive', 'positive', 'positive', ..., 'positive', 'neutral',
        'positive'], dtype=object),
 array([2, 2, 2, ..., 2, 1, 2]))

In [68]:
text = ['I love the hamburger you cook. Actually, you must add more garlic to make it better']
text_features = tfidf_vect.transform(text)
predictions = model.predict(text_features)
predictions

array([2])

array([2])